In [1]:
with open("test_input_1.txt") as fin:
    blocks = [[line.strip() for line in block.split("\n")] for block in "".join(fin).split("\n\n")]

print("\n\n".join("\n".join(b) for b in blocks))

#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#


## Part 1

The idea is as follows: We maintain a set of possible "mirror candidates", from which we continuously eliminate. At the first step, each column is a candidate. 
For each line, we check whether a candidate's reflection does still match. If it does not, we eliminate this candidate.

Let's limit our considerations to a column on the left half of the string - for the right half, it is the same but on the reversed string. In this case, we have to measure the distance between our chosen column and the closest end of the string (in this case the left end):

```
123456789
#.##..##.
  ><
```

In our example, we consider column 3 as a mirror candidate. The distance to the left end is 3 and the substring to this end is "#.#". Now, we consider the **reverse** of the right substring of the same length starting at the same position, i.e. "..#"). These two do not match:
```
#.#
..#
^
``` 
Therefore, we can eliminate column number 3 from our set of candidates and proceed. We repeat this process for every other candidate. When we iterated over all candidates, we move to the next line in order to eliminate more candidates.

When once processed all lines, there should be at most one candidate left.

In [2]:
def count_mismatches(line, column):
    if column < len(line)/2:
        # If we are in the first half if the line, 
        # we have to consider the distance to the left end
        # and derive the respective substrings
        left_side = line[:column]
        reversed_right_side = line[column:2*column][::-1]
    else:
        # If we are in the second half if the line, 
        # we have to consider the distance to the right end
        # and derive the respective substrings
        left_side = line[2*column-len(line):column]
        reversed_right_side = line[column:][::-1]
    
    # Count the mismatches (we need this for task 2)
    return sum(1 for l,r in zip(left_side, reversed_right_side) if l != r)

example_line = "#.##..##."
print(f"Mismatches on \"{example_line}\":")
for candidate in range(1,len(example_line)):
    print(f"Candidate column {candidate}:",count_mismatches(example_line,candidate))

Mismatches on "#.##..##.":
Candidate column 1: 1
Candidate column 2: 1
Candidate column 3: 1
Candidate column 4: 3
Candidate column 5: 0
Candidate column 6: 3
Candidate column 7: 0
Candidate column 8: 1


As we can see, there would be only two candidates left after processing the first line of our example input: $\{5,7\}$

In [3]:
def find_mirrors(lines):
    width = len(lines[0])
    # create a set of mirror candidates
    candidates = set(range(1,width))
    for line in lines:
        new_candidates = candidates.copy()
        for c in candidates:
            # If they are not equal, we eliminate them from the set of candidates
            if not count_mismatches(line, c) == 0:
                new_candidates.remove(c)
        candidates = new_candidates
    return candidates

The algorithm above does only work for columns. But we can easily use the same method by transposing the input (i.e. swapping columns and rows)
The solution is then the position of all horizontal mirrors plus 100 times the position all vertical mirrors.

In [4]:
def solve(blocks, finder):
    s = 0
    for lines in blocks:
        horizontal_mirrors = finder(lines)
        transposed_lines = [[lines[j][i] for j in range(len(lines))] for i in range(len(lines[0]))]  
        vertical_mirrors = finder(transposed_lines)
        s += sum(100*v for v in vertical_mirrors) + sum(horizontal_mirrors)
    return s

print(solve(blocks, find_mirrors))

405


## Part 2

This approach follows the same general idea. But this time we maintain two sets. The candidate set from the previous solution tracked all mirror candidates - i.e. those columns for which the corresponding substrings did match in each row. Now, we add a second set: The candidates with a single mismatch, which starts empty. When we consider a candidate without prior mismatch (i.e. from the first set) and we detect a single mismatch between the respective substrings, we still remove it from the first set, but we add it to the second one. We do not do the same, if we detect a larger mismatch!

Further, we also have to check the candidates in the second set. If we detect another mismatch in any other line, we discard this candidate entirely.

In [5]:
def find_mirrors2(lines):
    width = len(lines[0])
    candidates_without_mismatch = set(range(1,width))
    candidates_with_singular_mismatch = set()
    for lineno, line in enumerate(lines):
        new_candidates_without_change = candidates_without_mismatch.copy()
        new_candidates_with_change = candidates_with_singular_mismatch.copy()
        for c in candidates_without_mismatch:
            mismatches = count_mismatches(line, c)
            if not mismatches == 0:
                new_candidates_without_change.remove(c)
                # Did we detect a single mismatch?
                if mismatches == 1:
                    # Move candidate to second set
                    new_candidates_with_change.add(c)
        for c in candidates_with_singular_mismatch:
            # If we detect another mismatch for a candidate that had a single mismatch,
            # we discard it entirely.
            mismatches = count_mismatches(line, c)
            if not mismatches == 0:
                new_candidates_with_change.remove(c)
        candidates_with_singular_mismatch = new_candidates_with_change
        candidates_without_mismatch = new_candidates_without_change
    # Our result requires there to be a single mismatch
    return candidates_with_singular_mismatch


print("Output:", solve(blocks, find_mirrors2))

Output: 400
